# 04: Advanced Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/04_advanced_analysis.ipynb)

This notebook covers advanced snowpack analysis techniques including stability indices, hazard calculations, and using xsnow extensions.

## What You'll Learn

- Stability indices and their calculation
- Hazard chart calculations
- Critical crack length
- Comparing multiple locations and scenarios
- Advanced temporal analysis
- Using xsnow extensions


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



In [ ]:
import xsnow
import os
import numpy as np
import matplotlib.pyplot as plt

# Load xsnow's lightweight sample time series dataset
print("Loading xsnow sample data for advanced analysis...")
print("Using xsnow.single_profile_timeseries()")

try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded for advanced analysis")
    print(f"Dataset dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    ds = None


## Part 1: Stability Indices

Stability indices help assess avalanche risk. Common ones include:
- **Skier Stability Index (SK38)**: Probability of skier-triggered avalanche
- **Stability Index**: General stability metric
- **Rutschblock Score**: Field test equivalent

**Note**: These calculations depend on specific formulas and may require xsnow extensions or custom implementation. Let's explore what's available:


In [ ]:
# Check if xsnow has built-in stability index functions
# Check for extension methods
if ds is not None:
    extension_methods = [attr for attr in dir(ds) if not attr.startswith('_') and 
                        ('stability' in attr.lower() or 'hazard' in attr.lower() or 
                         'sk38' in attr.lower() or 'crack' in attr.lower())]
    
    if extension_methods:
        print(f"Found extension methods: {extension_methods}")
        print("Check xsnow documentation for usage instructions.")
    else:
        print("No built-in stability index methods found.")
        print("You may need to compute stability indices manually or use xsnow extensions.")
    
    # Example: Simple stability proxy based on density gradient
    # Compute density gradient (change in density with depth)
    # Large gradients can indicate weak layers
    if 'density' in ds.data_vars:
        density_gradient = ds['density'].diff(dim='layer')
        print(f"\nDensity gradient computed. Shape: {density_gradient.shape}")
        print("Large gradients may indicate weak layers.")
else:
    print("Dataset not loaded. Load data first.")



## Part 2: Temperature Gradient Analysis

Temperature gradients are important for understanding metamorphism and weak layer formation:


In [ ]:
# Temperature gradient = change in temperature per unit depth
# Large gradients can lead to faceting (weak layer formation)

if ds is not None and 'temperature' in ds.data_vars and 'z' in ds.coords:
    # Compute gradient using z coordinate
    temp_diff = ds['temperature'].diff(dim='layer')
    z_diff = ds.coords['z'].diff(dim='layer')
    
    # Avoid division by zero
    temp_gradient = temp_diff / z_diff.where(z_diff != 0, np.nan)
    
    # Or use the temperature_gradient if already computed
    if 'temperature_gradient' in ds.data_vars:
        tg = ds['temperature_gradient']
        print("Using pre-computed temperature gradient")
    else:
        tg = temp_gradient
        print("Computed temperature gradient from temperature and z")
    
    # Find layers with high gradients (> 10 K/m often indicates faceting)
    high_gradient = tg > 10  # K/m
    n_high_gradient = high_gradient.sum().values
    
    print(f"Found {n_high_gradient} layers with temperature gradient > 10 K/m")
    print("High gradients may indicate faceting conditions (weak layer formation)")
else:
    print("""
    Temperature gradient analysis:
    
    # Compute gradient
    temp_gradient = ds['temperature'].diff(dim='layer') / ds.coords['z'].diff(dim='layer')
    
    # Or use pre-computed gradient
    if 'temperature_gradient' in ds.data_vars:
        tg = ds['temperature_gradient']
    
    # Find high-gradient layers (faceting conditions)
    high_gradient = tg > 10  # K/m threshold
    """)


## Part 3: Hazard Chart Calculations

Hazard charts plot stability indices against depth. xsnow may have extensions for this:


In [ ]:
# Check for hazard chart extension
# Look for hazard-related methods
if ds is not None:
    if hasattr(ds, 'compute_hazard_chart') or hasattr(ds, 'hazard_chart'):
        print("Hazard chart extension found. Check xsnow documentation for usage.")
        # Example usage would be:
        # hc = ds.compute_hazard_chart()
    else:
        print("No built-in hazard chart method. Creating manual example...")
    
    # Manual example: Create a simple stability-depth plot
    # Simple proxy: use inverse density as stability proxy
    # (lower density = potentially weaker, but this is simplified!)
    profile = ds.isel(location=0, time=-1, slope=0, realization=0)  # Last time step
    
    if 'z' in profile.coords and 'density' in profile.data_vars:
        depth = -profile.coords['z'].values
        density = profile['density'].values
        
        # Simple stability proxy (NOT a real stability index!)
        stability_proxy = 1.0 / (density + 1.0)  # Avoid division by zero
        
        fig, ax = plt.subplots(figsize=(8, 10))
        ax.plot(stability_proxy, depth, 'b-', linewidth=2)
        ax.set_xlabel('Stability Proxy (1/density)', fontsize=12)
        ax.set_ylabel('Depth (m)', fontsize=12)
        ax.set_title('Simple Stability-Depth Profile\n(Example - Not Real Stability Index)', 
                    fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.invert_yaxis()
        plt.tight_layout()
        plt.show()
    else:
        print("Required variables (z, density) not available for stability plot")
else:
    print("Dataset not loaded.")


## Part 4: Critical Crack Length

Critical crack length is an advanced metric for assessing avalanche propagation potential:


In [ ]:
# Check for critical crack length method
if ds is not None:
    if hasattr(ds, 'compute_critical_crack_length'):
        print("Critical crack length method found.")
        # Example: ccl = ds.compute_critical_crack_length()
    else:
        print("No built-in critical crack length method.")
        print("This is an advanced metric that may require xsnow extensions or custom implementation.")
        print("Check xsnow documentation for available extensions.")
else:
    print("Dataset not loaded.")


## Part 5: Comparing Multiple Locations


In [ ]:
if ds is not None:
    n_locations = ds.dims.get('location', 0)
    
    if n_locations > 1:
        print(f"Comparing {n_locations} locations:")
        
        # Compare mean density across locations
        if 'density' in ds.data_vars:
            mean_density_by_location = ds['density'].mean(dim=['time', 'layer'])
            print("\nMean density by location:")
            for i, loc in enumerate(ds.coords['location'].values):
                mean_dens = mean_density_by_location.isel(location=i).values
                print(f"  {loc}: {mean_dens:.1f} kg/m³")
        
        # Compare snow height evolution
        if 'HS' in ds.data_vars:
            print("\nSnow height by location (last time step):")
            for i, loc in enumerate(ds.coords['location'].values):
                hs_loc = ds['HS'].isel(location=i, slope=0, realization=0, time=-1)
                print(f"  {loc}: {hs_loc.values:.2f} m")
    else:
        print("Only one location in dataset. To compare multiple locations:")
        print("  ds = xsnow.read(['data/station1.pro', 'data/station2.pro'])")
else:
    print("""
    Comparing multiple locations:
    
    # Load multiple files
    ds = xsnow.read(['data/station1.pro', 'data/station2.pro'])
    
    # Compare statistics
    mean_density = ds['density'].mean(dim=['time', 'layer'])
    
    # Compare time series
    for loc in ds.coords['location']:
        hs = ds['HS'].sel(location=loc)
        # Plot or analyze
    """)


## Part 6: Advanced Temporal Analysis


In [ ]:
# Resample to daily (if data is hourly)
# This averages hourly data to daily
if ds is not None:
    try:
        ds_daily = ds.resample(time='1D').mean()
        print(f"Resampled to daily: {dict(ds_daily.dims)}")
    except Exception as e:
        print(f"Could not resample (may already be daily or different frequency): {e}")
    
    # Rolling window statistics
    if 'HS' in ds.data_vars:
        hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
        
        # 7-day rolling average
        try:
            hs_7day = hs_series.rolling(time=7, center=True).mean()
            print("7-day rolling average computed successfully")
        except Exception as e:
            print(f"Could not compute rolling average: {e}")
        
        # Rate of change
        hs_rate = hs_series.diff(dim='time')
        print(f"Rate of change computed. Mean: {hs_rate.mean().values:.4f} m/time_step")
else:
    print("Dataset not loaded.")


## Part 7: Using xsnow Extensions


In [ ]:
# Check what extensions/methods are available
methods = [m for m in dir(ds) if not m.startswith('_') and callable(getattr(ds, m, None))]

# Filter for potentially interesting methods
interesting = [m for m in methods if any(keyword in m.lower() for keyword in 
                                        ['compute', 'calculate', 'hazard', 'stability', 
                                         'crack', 'classify', 'mask'])]

if interesting:
    for m in interesting[:10]:  # Show first 10
else:
    xsnow extensions:
    
    # Extensions add methods to xsnowDataset
    # Common extensions include:
    # - Classification (mask_by_criteria, etc.)
    # - Hazard charts
    # - Stability indices
    # - Critical crack length
    
    # Check xsnow docs for:
    # - How to import extensions
    # - Available extension methods
    # - How to create custom extensions (see notebook 06)
    """)


## Part 8: Ensemble Analysis

If you have multiple realizations (ensemble runs), analyze uncertainty:


In [ ]:
n_realizations = ds.dims.get('realization', 0)

if n_realizations > 1:
    
    if 'HS' in ds.data_vars:
        # Compute statistics across realizations
        hs_mean = ds['HS'].mean(dim='realization')
        hs_std = ds['HS'].std(dim='realization')
        hs_min = ds['HS'].min(dim='realization')
        hs_max = ds['HS'].max(dim='realization')
        
else:
    Ensemble analysis:
    
    # Load multiple realizations
    ds = xsnow.read(['run1.pro', 'run2.pro', 'run3.pro'])
    # Or ensure realization dimension has multiple values
    
    # Statistics across ensemble
    mean = ds['HS'].mean(dim='realization')
    std = ds['HS'].std(dim='realization')
    min_val = ds['HS'].min(dim='realization')
    max_val = ds['HS'].max(dim='realization')
    
    # Plot with uncertainty bands
    # plt.fill_between(times, min_val, max_val, alpha=0.3)
    """)


## Summary

✅ **What we learned:**

1. **Stability indices**: Methods for assessing avalanche risk (may require extensions)
2. **Temperature gradients**: Important for understanding weak layer formation
3. **Hazard charts**: Visualizing stability vs depth
4. **Critical crack length**: Advanced propagation metric
5. **Multi-location comparison**: Analyzing spatial patterns
6. **Temporal analysis**: Resampling, rolling windows, rates of change
7. **Extensions**: Using xsnow's extension system
8. **Ensemble analysis**: Working with multiple realizations

## Key Advanced Techniques

- **Resampling**: `ds.resample(time='1D').mean()` for temporal aggregation
- **Rolling windows**: `ds.rolling(time=7).mean()` for smoothing
- **Ensemble stats**: `.mean(dim='realization')`, `.std(dim='realization')`
- **Extensions**: Check xsnow docs for available extension methods

## Next Steps

Ready to work with your own data? Move on to:
- **05_working_with_custom_data.ipynb**: Load and analyze your own files

Or learn to extend xsnow:
- **06_extending_xsnow.ipynb**: Create custom analysis functions

## Exercises

1. Compute temperature gradient and identify layers with gradients > 10 K/m
2. Resample hourly data to daily and compare statistics
3. If you have multiple locations, compare their mean density profiles
4. Calculate a 7-day rolling average of snow height
5. Check xsnow documentation for available extensions and try one
